In [3]:
import cv2
import os
from tensorflow.keras.models import load_model
import imutils
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [4]:
protxtPath = r"pretrained-models\\deploy.prototxt"
weightsPath = r"pretrained-models\\res10_300x300_ssd_iter_140000.caffemodel"

In [5]:
faceNet = cv2.dnn.readNet(protxtPath, weightsPath)

In [6]:
maskNet = load_model("mask_detector.model")

In [14]:
def detect_and_predict_masks(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1, (224, 224), (104, 177, 123))
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections)
    
    faces, locs, preds = [], [], []
    
    for i in range(0, detections.shape[2]):
        
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startx, starty, endx, endy) = box.astype('int')
            
            (startx, starty) = (max(0, startx), max(0, starty))
            (endx, endy) = (max(0, endx), max(0, endy))
            
            face = frame[starty:endy, startx:endx]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startx, starty, endx, endy))
            
#         if len(faces) > 0:
#             faces = np.array(faces, dtype='float32')
#             preds = maskNet.predict(faces, batch_size=32)
    return locs, preds        

In [15]:
vs = cv2.VideoCapture(0)
while True:
    ret, frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    (locs, preds) = detect_and_predict_masks(frame, faceNet, maskNet)
    
    for (box, preds) in zip(locs, preds):
        (startx, starty, endx, endy) = box
        (mask, withoutMask) = [0, 0]
        
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        cv2.putText(frame, label, (startx, starty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startx, starty), (endx, endy), color, 2)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        vs.release()
        cv2.destroyAllWindows()
        break

[[[[0.         1.         0.9759321  ... 0.28454384 0.7013809
    0.8245344 ]
   [0.         1.         0.1327465  ... 4.0235453  5.1378202
    5.3383675 ]
   [0.         1.         0.12085351 ... 4.025283   1.1047728
    5.345049  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.97197604 ... 0.28731775 0.7017165
    0.82942927]
   [0.         1.         0.13212416 ... 4.0239472  5.1373744
    5.3394027 ]
   [0.         1.         0.12030325 ... 4.0261016  1.1052425
    5.3445067 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.95904213 ... 0.2862269  0.70168084
    0.8375759 ]
   [0. 

[[[[0.         1.         0.9700721  ... 0.2872197  0.7023572
    0.84338063]
   [0.         1.         0.13174048 ... 4.023973   5.137266
    5.338899  ]
   [0.         1.         0.11966486 ... 4.026535   1.1049056
    5.344017  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.97726214 ... 0.29067597 0.7021588
    0.8384527 ]
   [0.         1.         0.1316241  ... 4.0238976  5.1372037
    5.3392425 ]
   [0.         1.         0.11970586 ... 4.0263195  1.103932
    5.3447676 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9806246  ... 0.29186538 0.70166284
    0.8386884 ]
   [0.   

[[[[0.         1.         0.89383507 ... 0.3196752  0.7196934
    0.856693  ]
   [0.         1.         0.13057673 ... 4.0243278  5.137514
    5.3379207 ]
   [0.         1.         0.12130122 ... 4.0248003  1.1057477
    5.344042  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.89255816 ... 0.31413156 0.7276603
    0.8591867 ]
   [0.         1.         0.1309371  ... 4.024022   5.1372666
    5.3379526 ]
   [0.         1.         0.1214096  ... 4.0243955  1.1052091
    5.3444257 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.89092135 ... 0.31086665 0.7305384
    0.85861427]
   [0.   

[[[[0.         1.         0.9891672  ... 0.32325795 0.75429714
    0.8901602 ]
   [0.         1.         0.13079783 ... 4.0241656  5.136809
    5.337168  ]
   [0.         1.         0.12341901 ... 4.0236073  1.1067569
    5.344331  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9922208  ... 0.32420087 0.75728714
    0.8895502 ]
   [0.         1.         0.13039826 ... 4.0244865  5.1367965
    5.3371954 ]
   [0.         1.         0.12323081 ... 4.0240564  1.1064997
    5.3442717 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9953681  ... 0.3195492  0.7599628
    0.8972745 ]
   [0. 

[[[[0.         1.         0.9976331  ... 0.32977927 0.76781523
    0.9036964 ]
   [0.         1.         0.13171773 ... 4.0232983  5.135496
    5.3368263 ]
   [0.         1.         0.12379187 ... 4.023928   1.1065401
    5.3441596 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99795985 ... 0.3284234  0.76752263
    0.8986927 ]
   [0.         1.         0.13116722 ... 4.0236883  5.1356826
    5.336768  ]
   [0.         1.         0.12362567 ... 4.0237846  1.1063644
    5.3441477 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9973972  ... 0.3301236  0.76553434
    0.9006807 ]
   [0.

...

[[[[0.         1.         0.9883229  ... 0.37034532 0.8607313
    0.94657135]
   [0.         1.         0.12979622 ... 4.021618   5.1346016
    5.329624  ]
   [0.         1.         0.12378605 ... 4.025806   1.1028731
    5.344785  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9917697  ... 0.36400545 0.8582058
    0.9511548 ]
   [0.         1.         0.13076037 ... 4.0208855  5.1340213
    5.3295918 ]
   [0.         1.         0.12364653 ... 4.0260124  1.1027501
    5.3448763 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9961777  ... 0.36697388 0.84485453
    0.94257355]
   [0. 

[[[[0.         1.         0.9985808  ... 0.32574892 0.6801005
    0.88039994]
   [0.         1.         0.13332994 ... 4.0225077  5.1350713
    5.33718   ]
   [0.         1.         0.11653543 ... 0.03279257 5.138891
    1.3154106 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.999091   ... 0.3176427  0.6733482
    0.8853135 ]
   [0.         1.         0.1336234  ... 4.022074   5.134475
    5.3379016 ]
   [0.         1.         0.11641656 ... 0.03233331 5.1386356
    1.3154614 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99904627 ... 0.31619453 0.6717243
    0.8775257 ]
   [0.    

[[[[0.         1.         0.98268664 ... 0.3036421  0.672801
    0.8586194 ]
   [0.         1.         0.13281606 ... 4.023011   5.1370344
    5.338828  ]
   [0.         1.         0.11766387 ... 4.0275345  1.1037223
    5.3440123 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9759445  ... 0.3014624  0.6753298
    0.8479806 ]
   [0.         1.         0.13245368 ... 4.023288   5.1372375
    5.33906   ]
   [0.         1.         0.11776116 ... 4.027464   1.1039665
    5.3441544 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.983679   ... 0.29902646 0.67989254
    0.85433507]
   [0.  

[[[[0.         1.         0.9596023  ... 0.31210563 0.69430196
    0.861233  ]
   [0.         1.         0.13132253 ... 4.0237927  5.13681
    5.3390985 ]
   [0.         1.         0.11961836 ... 4.0261207  1.1044225
    5.3438525 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9602464  ... 0.31033057 0.6965845
    0.8563127 ]
   [0.         1.         0.13108449 ... 4.0239224  5.137128
    5.3389316 ]
   [0.         1.         0.11959652 ... 4.026103   1.1045913
    5.343979  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9497913  ... 0.31075266 0.6940899
    0.8590535 ]
   [0.    

[[[[0.         1.         0.9760282  ... 0.34016487 0.7384074
    0.8748827 ]
   [0.         1.         0.13169755 ... 4.0248194  5.137339
    5.3376427 ]
   [0.         1.         0.12238808 ... 4.024326   1.106898
    5.344442  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9820915  ... 0.3421882  0.74077904
    0.8788171 ]
   [0.         1.         0.13137427 ... 4.0250306  5.137253
    5.337543  ]
   [0.         1.         0.12292612 ... 4.0241838  1.1072642
    5.3441043 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.98746735 ... 0.3458411  0.73960036
    0.86662   ]
   [0.   

In [13]:
vs.release()
cv2.destroyAllWindows()